In [1]:
# load required packages
from keras.applications import *
from keras.applications.vgg16 import *
from keras.models import *
from keras.preprocessing import image
from keras.preprocessing.image import *
from keras.layers import Dense,Flatten,GlobalAveragePooling2D,Dropout,BatchNormalization
import h5py as h
import numpy as np
import os
import glob
import shutil
from keras import backend as K

from keras.utils import to_categorical
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import math
from utils import *

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ec2-user/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ec2-user/.config/matplotlib/matplotli

In [2]:
train_path = '/home/ec2-user/dogs_cats/train/'
valid_path = '/home/ec2-user/dogs_cats/valid/'
test_path = '/home/ec2-user/dogs_cats/test/'

In [3]:
batch_size = 128

In [35]:
#preprocess_data
trn_args = dict(preprocessing_function=xception.preprocess_input,
               zoom_range = 0.2, #0.2,0.3
               rotation_range = 20, 
                shear_range = 0.2,
             width_shift_range = 0.2,
          height_shift_range = 0.2,
             horizontal_flip = True,
            channel_shift_range = 0.2
                
               )
test_args = dict(preprocessing_function=xception.preprocess_input)
trn_gen = preprocess_data(trn_args)
test_gen = preprocess_data(test_args)

In [36]:
#load data by generator
train_generator = get_data(trn_gen,train_path,input_size=299,batch_size=batch_size)
valid_generator = get_data(test_gen,valid_path,input_size=299,batch_size=batch_size)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [37]:
# one-hot label
trn_label = to_categorical(train_generator.classes,num_classes=2)
val_label = to_categorical(valid_generator.classes,num_classes=2)

In [38]:
# compute steps
trn_steps = comput_step(train_generator.n,batch_size)
val_steps = comput_step(valid_generator.n,batch_size)

In [8]:
# create pre-trained 
base_model = Xception(weights='imagenet',include_top=False,pooling = 'avg')

In [10]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [9]:
x = base_model.output
#x = BatchNormalization()(x)
x = Dropout(0.6)(x)
pred = Dense(2,activation = 'softmax')(x)
model = Model(inputs=base_model.input,outputs=pred)

In [10]:
for layer in base_model.layers:layer.trainable=False

In [17]:
for layer in base_model.layers[-4:]:layer.trainable = True

In [18]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [16]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 6 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [40]:
model.optimizer.lr = 1e-6

In [41]:
model.load_weights('Xception_FT.h5')

In [43]:
model.fit_generator(train_generator,epochs=1,validation_data=valid_generator,
                    steps_per_epoch=trn_steps,validation_steps=val_steps,workers=5)

Epoch 1/1
180/180 [==============================] - 133s 740ms/step - loss: 0.0225 - acc: 0.9919 - val_loss: 0.0258 - val_acc: 0.9930


In [35]:
model.fit_generator(train_generator,epochs=3,validation_data=valid_generator,
                    steps_per_epoch=trn_steps,validation_steps=val_steps,workers=5)

Epoch 1/3
180/180 [==============================] - 128s 713ms/step - loss: 0.0628 - acc: 0.9805 - val_loss: 0.0205 - val_acc: 0.9940
Epoch 2/3
180/180 [==============================] - 120s 669ms/step - loss: 0.0287 - acc: 0.9895 - val_loss: 0.0229 - val_acc: 0.9930
Epoch 3/3
180/180 [==============================] - 121s 673ms/step - loss: 0.0278 - acc: 0.9898 - val_loss: 0.0230 - val_acc: 0.9945


In [21]:
#unfreeze -6: layer

In [52]:
model.save_weights('Xception_FT.h5')

OSError: Unable to create file (unable to truncate a file which is already open)

In [22]:
test_generator = get_test_data(test_gen,test_path,input_size=299,batch_size=batch_size)

Found 12500 images belonging to 1 classes.


In [23]:
test_steps = comput_step(test_generator.n,batch_size)

In [24]:
test_features = model.predict_generator(test_generator,verbose=1)

98/98 [==============================] - 64s 657ms/step


In [44]:
pred_2 = model.predict_generator(test_generator,verbose=1)

98/98 [==============================] - 54s 547ms/step


In [54]:
pred = pred_2.clip(min=0.005,max=0.99)

In [55]:
filenames = test_generator.filenames

In [56]:
is_dog = pred[:,1]

In [57]:
index = [int(fname[fname.rfind('/')+1:fname.rfind('.')]) for fname in filenames]

In [58]:
sub_xception = np.stack([index,is_dog],axis=1)

In [59]:
from pandas import DataFrame
import pandas as pd

In [60]:
submission_xception = DataFrame(sub_xception,columns=['id','label'])

In [61]:
submission_xception.id = submission_xception.id.astype(int)

In [62]:
submission_xception.to_csv('submission_xception2.csv',index=None)